In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox, Layout
from scipy.optimize import curve_fit

# Load the experimental image once from the .npy file
exp_image = np.load('/Users/xiaodong/Downloads/glycine_frame4/glycine_frame4.npy')
# exp_image = np.load('/home/bubl3932/files/glycine/batch_1640000_1649999/glycine_frame4.npy')

def pseudo_voigt(xx, yy, x0, y0, amplitude, sigma, gamma, eta):
    """
    Generate a pseudo Voigt profile.
    - Gaussian: exp(-((x-x0)^2+(y-y0)^2)/(2*sigma^2))
    - Lorentzian: 1/(1+((x-x0)^2+(y-y0)^2)/gamma^2)
    - eta: mixing factor (0 => pure Gaussian, 1 => pure Lorentzian)
    """
    r2 = (xx - x0)**2 + (yy - y0)**2
    gaussian = np.exp(-r2 / (2 * sigma**2))
    lorentzian = 1 / (1 + r2 / gamma**2)
    return amplitude * (eta * lorentzian + (1 - eta) * gaussian)

def simulate_image(nx, ny, beam_center_x, beam_center_y, amplitude, sigma, gamma, eta,
                   background, poisson_level, read_noise_std):
    """
    Create a simulated 2D diffraction image based on a pseudo-Voigt profile plus noise.
    """
    # Create coordinate grids
    x = np.arange(nx)
    y = np.arange(ny)
    xx, yy = np.meshgrid(x, y)

    # Generate simulated beam
    beam = pseudo_voigt(xx, yy, beam_center_x, beam_center_y, amplitude, sigma, gamma, eta)

    # Add noise components
    noise_poisson = np.random.poisson(poisson_level, size=(ny, nx))
    noise_read = np.random.normal(loc=0, scale=read_noise_std, size=(ny, nx))

    image = beam + background + noise_poisson + noise_read
    image[image < 0] = 0  # ensure no negatives
    return image

def radial_profile(data, center, r_min=5):
    """
    Compute the radial profile of a 2D array and mask out data below r_min.
    
    Returns:
      r_i         : Radii (pixels) for which the profile is computed (r >= r_min)
      radial_mean : Average intensity in annular bins (for r >= r_min)
    """
    y, x = np.indices(data.shape)
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r_int = r.astype(int)

    tbin = np.bincount(r_int.ravel(), data.ravel())
    nr = np.bincount(r_int.ravel())

    radial_mean = tbin / nr
    r_i = np.arange(len(radial_mean))
    mask = r_i >= r_min

    return r_i[mask], radial_mean[mask]

def pseudo_voigt_1d(r, amplitude, sigma, gamma, eta, baseline):
    """
    1D pseudo Voigt function for fitting the radial profile.
    """
    gaussian = np.exp(-r**2 / (2 * sigma**2))
    lorentzian = 1 / (1 + (r / gamma)**2)
    return amplitude * (eta * lorentzian + (1 - eta) * gaussian) + baseline

def update_plots(amplitude, sigma, gamma, eta, background, poisson_level, read_noise_std,
                 beam_center_x, beam_center_y, exp_radial_center_x, exp_radial_center_y,
                 xmin_axis):
    """
    Update all plots (radial profile + images) whenever a parameter changes.
    """
    nx, ny = 512, 512

    # Generate simulated image
    sim_img = simulate_image(nx, ny, beam_center_x, beam_center_y, amplitude,
                             sigma, gamma, eta, background, poisson_level, read_noise_std)

    # Compute radial profiles (for simulation and experimental data)
    sim_center = (beam_center_x, beam_center_y)
    r_sim, profile_sim = radial_profile(sim_img, sim_center)

    exp_center = (exp_radial_center_x, exp_radial_center_y)
    r_exp, profile_exp = radial_profile(exp_image, exp_center)

    # Fit the experimental radial profile using pseudo_voigt_1d
    try:
        # Initial guess: amplitude from data range, sigma & gamma ~10, eta 0.5, baseline from min intensity.
        p0 = [max(profile_exp) - min(profile_exp), 10, 10, 0.5, min(profile_exp)]
        popt, pcov = curve_fit(pseudo_voigt_1d, r_exp, profile_exp, p0=p0)
        fitted_curve = pseudo_voigt_1d(r_exp, *popt)
        fit_text = (f"Amplitude: {popt[0]:.2f}\n"
                    f"Sigma: {popt[1]:.2f}\n"
                    f"Gamma: {popt[2]:.2f}\n"
                    f"Eta: {popt[3]:.2f}\n"
                    f"Baseline: {popt[4]:.2f}")
    except Exception as e:
        popt = None
        fitted_curve = None
        fit_text = "Fit failed."

    # --- Update the radial profile plot ---
    with out_radial:
        out_radial.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6,6))

        # Plot the simulation and experimental profiles
        ax.plot(r_sim, profile_sim, 'b-', label=f"Simulated\n(center={beam_center_x},{beam_center_y})")
        ax.plot(r_exp, profile_exp, 'r--', label=f"Experimental\n(center={exp_radial_center_x},{exp_radial_center_y})")

        # Plot the fitted curve if successful
        if fitted_curve is not None:
            ax.plot(r_exp, fitted_curve, 'k:', label="Fit")
            ax.text(0.05, 0.95, fit_text, transform=ax.transAxes, fontsize=10,
                    verticalalignment='top', bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5))

        ax.set_title("Radial Profiles & Fit")
        ax.set_xlabel("Radius (pixels)")
        ax.set_ylabel("Average Intensity")
        ax.legend()
        ax.grid(True)

        # 1) Lock the left bound on the x-axis
        ax.set_xlim(left=xmin_axis)

        # 2) Filter the data for r >= xmin_axis so we can get correct y-limits
        sim_mask = r_sim >= xmin_axis
        exp_mask = r_exp >= xmin_axis

        visible_sim = profile_sim[sim_mask]
        visible_exp = profile_exp[exp_mask]

        # Include fitted curve in the visible data if it exists
        if fitted_curve is not None:
            visible_fit = fitted_curve[exp_mask]
        else:
            visible_fit = np.array([])

        # Combine the visible data
        all_visible = np.concatenate([arr for arr in [visible_sim, visible_exp, visible_fit] if arr.size > 0])

        # 3) If there is visible data, adjust the y-limits accordingly
        if all_visible.size > 0:
            y_min = all_visible.min()
            y_max = all_visible.max()
            margin = (y_max - y_min) * 0.1 if y_max > y_min else 0.1
            ax.set_ylim(y_min - margin, y_max + margin)

        plt.show()

    # --- Update the image plots ---
    # Use the same intensity scale as the experimental image
    vmin = exp_image.min()
    vmax = exp_image.max()

    with out_images:
        out_images.clear_output(wait=True)
        fig, axs = plt.subplots(1, 2, figsize=(12,6))

        im0 = axs[0].imshow(sim_img, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
        axs[0].set_title("Simulated Diffraction")
        axs[0].set_xlabel("Pixel (fs)")
        axs[0].set_ylabel("Pixel (ss)")
        fig.colorbar(im0, ax=axs[0], label='Intensity')

        im1 = axs[1].imshow(exp_image, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
        axs[1].set_title("Experimental Background (Frame 4)")
        axs[1].set_xlabel("Pixel (fs)")
        axs[1].set_ylabel("Pixel (ss)")
        fig.colorbar(im1, ax=axs[1], label='Intensity')

        plt.tight_layout()
        plt.show()


# --- Slider Controls ---

# Use FloatSlider where fractional steps are needed.
amp_slider = widgets.IntSlider(value=10000, min=0, max=10000, step=100, description='Amplitude')
sigma_slider = widgets.FloatSlider(value=1, min=0, max=10, step=0.1, description='Sigma')
gamma_slider = widgets.FloatSlider(value=10, min=0, max=10, step=0.1, description='Gamma')
eta_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Eta')

background_slider = widgets.FloatSlider(value=5, min=-10, max=10, step=0.1, description='Background')
poisson_slider = widgets.FloatSlider(value=5, min=0, max=10, step=0.1, description='Poisson')
read_noise_slider = widgets.FloatSlider(value=5, min=0, max=10, step=0.1, description='Read Noise')

xmin_axis_slider = widgets.IntSlider(value=0, min=0, max=256, step=10, description='Radial Profile X-axis minimum')

beam_center_x_slider = widgets.IntSlider(value=256, min=0, max=512, step=1, description='Beam Center X')
beam_center_y_slider = widgets.IntSlider(value=256, min=0, max=512, step=1, description='Beam Center Y')
exp_radial_center_x_slider = widgets.IntSlider(value=238, min=206, max=306, step=1, description='Exp Radial Center X')
exp_radial_center_y_slider = widgets.IntSlider(value=242, min=206, max=306, step=1, description='Exp Radial Center Y')

# --- Output Areas ---
out_radial = widgets.Output()
out_images = widgets.Output()

# --- Layout ---
controls = VBox([
    amp_slider, sigma_slider, gamma_slider, eta_slider,
    background_slider, poisson_slider, read_noise_slider,
    beam_center_x_slider, beam_center_y_slider,
    exp_radial_center_x_slider, exp_radial_center_y_slider,
    xmin_axis_slider
])

interactive_out = interactive_output(update_plots, {
    'amplitude': amp_slider,
    'sigma': sigma_slider,
    'gamma': gamma_slider,
    'eta': eta_slider,
    'background': background_slider,
    'poisson_level': poisson_slider,
    'read_noise_std': read_noise_slider,
    'beam_center_x': beam_center_x_slider,
    'beam_center_y': beam_center_y_slider,
    'exp_radial_center_x': exp_radial_center_x_slider,
    'exp_radial_center_y': exp_radial_center_y_slider,
    'xmin_axis': xmin_axis_slider
})

top_row = HBox([controls, out_radial], layout=Layout(align_items='flex-start'))
layout = VBox([top_row, out_images])
display(layout)

# Trigger an initial update
update_plots(
    amp_slider.value, sigma_slider.value, gamma_slider.value, eta_slider.value,
    background_slider.value, poisson_slider.value, read_noise_slider.value,
    beam_center_x_slider.value, beam_center_y_slider.value,
    exp_radial_center_x_slider.value, exp_radial_center_y_slider.value,
    xmin_axis_slider.value
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

########################################
# 1) Load experimental data & radial profile
########################################

# Path to your .npy file (adjust as needed)
exp_image = np.load('/Users/xiaodong/Downloads/glycine_frame4/glycine_frame4.npy')

def radial_profile(data, center, r_min=5):
    """
    Compute the radial profile of a 2D array and mask out data below r_min.
    
    Returns:
      r_i         : Radii (pixels) for which the profile is computed (r >= r_min)
      radial_mean : Average intensity in annular bins (for r >= r_min)
    """
    y, x = np.indices(data.shape)
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r_int = r.astype(int)
    
    # Sum intensity in each radial bin & count how many pixels are in that bin
    tbin = np.bincount(r_int.ravel(), data.ravel())
    nr = np.bincount(r_int.ravel())
    
    radial_mean = tbin / nr
    r_i = np.arange(len(radial_mean))
    mask = r_i >= r_min
    
    return r_i[mask], radial_mean[mask]

# Choose an experimental center (adjust as needed)
exp_center = (238, 242)
r_exp, profile_exp = radial_profile(exp_image, exp_center)

########################################
# 2) Define the multi-peak pseudo-Voigt model
########################################

def single_pv_1d(r, amplitude, center, sigma, gamma, eta):
    """
    1D pseudo-Voigt with a given center.
    amplitude : peak intensity
    center    : center in r-space
    sigma     : Gaussian width
    gamma     : Lorentzian half-width
    eta       : mixing fraction (0=Gaussian, 1=Lorentzian)
    """
    dr = r - center
    gauss = np.exp(-dr**2 / (2 * sigma**2))
    lorentz = 1 / (1 + (dr**2 / gamma**2))
    return amplitude * (eta * lorentz + (1 - eta) * gauss)

def three_peak_model(r,
                     A0, s0, g0, e0,       # Peak #1 at r=0 (direct beam)
                     A1, c1, s1, g1, e1,  # Peak #2 ~ r=70
                     A2, c2, s2, g2, e2,  # Peak #3 ~ r=130
                     baseline):
    """
    Sum of three pseudo-Voigt peaks, plus a constant baseline.
    - Peak 1 is fixed at center=0.
    - Peak 2 has a free center c1 (initial guess ~70).
    - Peak 3 has a free center c2 (initial guess ~130).
    """
    peak0 = single_pv_1d(r, A0, 0.0, s0, g0, e0)
    peak1 = single_pv_1d(r, A1, c1, s1, g1, e1)
    peak2 = single_pv_1d(r, A2, c2, s2, g2, e2)
    return peak0 + peak1 + peak2 + baseline

########################################
# 3) Fit the model to the experimental radial profile
########################################

# Provide initial guesses for all parameters:
# (A0, s0, g0, e0,
#  A1, c1, s1, g1, e1,
#  A2, c2, s2, g2, e2,
#  baseline)
p0 = [
    9000,  2.7,  7.5,  0.06,   # Peak#1 at r=0
    100,   70,  5,  5,  0.5,  # Peak#2 near 70
    100,   130, 5,  5,  0.5,  # Peak#3 near 130
    0    # Baseline
]

try:
    popt, pcov = curve_fit(three_peak_model, r_exp, profile_exp, p0=p0, maxfev=10000)
except RuntimeError:
    print("Fit did not converge; using initial guesses for final plot.")
    popt = p0

# Evaluate the best-fit model
fit_curve = three_peak_model(r_exp, *popt)

# Also get individual peaks for reference
peak0_fit = single_pv_1d(r_exp, popt[0], 0.0, popt[1], popt[2], popt[3])
peak1_fit = single_pv_1d(r_exp, popt[4], popt[5], popt[6], popt[7], popt[8])
peak2_fit = single_pv_1d(r_exp, popt[9], popt[10], popt[11], popt[12], popt[13])

########################################
# 4) Plot the result
########################################
plt.figure(figsize=(8,5))
plt.plot(r_exp, profile_exp, 'ko', label='Experimental Data')
plt.plot(r_exp, fit_curve,    'b-', label='Best Fit (3 peaks + baseline)')

# Show each peak offset by the baseline
baseline = popt[-1]
plt.plot(r_exp, peak0_fit + baseline, 'r--', label='Peak at r=0')
plt.plot(r_exp, peak1_fit + baseline, 'g--', label='Peak near r=70')
plt.plot(r_exp, peak2_fit + baseline, 'm--', label='Peak near r=130')

plt.xlabel('Radius (pixels)')
plt.ylabel('Intensity')
plt.title('Fit of Direct Beam + 2 Bumps using Pseudo-Voigt Peaks')
plt.grid(True)
plt.legend()
plt.show()

########################################
# 5) Print best-fit parameters
########################################
print("Best-Fit Parameters:")
print(f" Peak@r=0:       Amp={popt[0]:.2f}, sigma={popt[1]:.2f}, gamma={popt[2]:.2f}, eta={popt[3]:.2f}")
print(f" Peak@r={popt[5]:.2f}: Amp={popt[4]:.2f}, sigma={popt[6]:.2f}, gamma={popt[7]:.2f}, eta={popt[8]:.2f}")
print(f" Peak@r={popt[10]:.2f}:Amp={popt[9]:.2f}, sigma={popt[11]:.2f}, gamma={popt[12]:.2f}, eta={popt[13]:.2f}")
print(f" Baseline:        {popt[14]:.2f}")


ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2